<a href="https://colab.research.google.com/github/valuedriven/dataanalysis/blob/master/trabalho_final/trabalho_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Passos requeridos para o trabalho:
- abrir o conjunto de dados em um DataFrame, 
- visualizar os dados por meio de gráficos construídos com uma das bibliotecas gráficas vistas na disciplina,
- utilizar aprendizado de máquina não supervisionado e/ou supervisionado para construir um modelo que seja capaz de prever uma propriedade de interesse.